In [120]:
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn import metrics
import plotly.express as px
import autosklearn.classification
%matplotlib inline

FileNotFoundError: [Errno 2] No such file or directory: '/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scikit_learn-0.23.2.dist-info/METADATA'

In [ ]:
stay_at_home_df = pd.read_csv("dataset/Stay at home.csv")

In [ ]:
stay_at_home_df.head()

In [ ]:
cleaned_stay_at_home_df = stay_at_home_df[['location','date','total_cases','total_tests','stringency_index','positive_rate','Stay at home requirements (OxBSG)','total_deaths']]

In [ ]:
cleaned_stay_at_home_df

In [ ]:
cleaned_stay_at_home_df['cases_per_tests_ratio'] = cleaned_stay_at_home_df['total_cases']/cleaned_stay_at_home_df['total_tests']

In [ ]:
cleaned_stay_at_home_df

In [ ]:
dummies_column = pd.get_dummies(cleaned_stay_at_home_df['Stay at home requirements (OxBSG)'])

In [ ]:
cleaned_stay_at_home_df = cleaned_stay_at_home_df.join(dummies_column)

In [ ]:
cleaned_stay_at_home_df.head()

In [ ]:
cleaned_stay_at_home_df.dtypes

In [ ]:
groupby_location_df = cleaned_stay_at_home_df.groupby('location',as_index=False)
separated_location_df = dict(iter(groupby_location_df))

In [ ]:
def TimeSeriesTrainTestSplit(X, y, test_size):
    
        test_index = int(len(X)*(1-test_size))
        print('test_index',test_index)
        X_train = X.iloc[:test_index]
        y_train = y.iloc[:test_index]
        X_test = X.iloc[test_index:]
        y_test = y.iloc[test_index:]
        return X_train, y_train, X_test, y_test

def getScore(location_df):
    location_df = location_df.fillna(method ='ffill')
    X = location_df[[0,1,2,3,'total_tests','total_deaths','stringency_index']]
    y = location_df['total_cases']
    X_train, y_train, X_test, y_test = TimeSeriesTrainTestSplit(X,y, 0.2)
    regr = linear_model.LinearRegression() # Do not use fit_intercept = False if you have removed 1 column after dummy encoding
    print(X_train.shape, y_train.shape)
    regr.fit(X_train,y_train)
    model_score = regr.score(X_test,y_test)
    print(model_score)
    cls = autosklearn.classification.AutoSklearnClassifier()
    cls.fit(X_train, y_train)
    predictions = cls.predict(X_test)
    print(predictions)

In [ ]:
location_model_score = {}
for location in separated_location_df: 
    print(location)
    getScore(separated_location_df[location])
    
